# changing the order of the picks so they will be read in correctly by NLLocLib.c


In [1]:
import os
import sys
import glob
import pandas as pd
import numpy as np
from obspy import read
from obspy.core import UTCDateTime

In [2]:
# path='/Users/banjo/Projects/nll_pnsn/util/phaseFiles'
# ifile='/pnsn20.hypo' 
# # ifile='/or80.hypo'
# ofile = '/Users/banjo/Projects/nll_pnsn/util/stationInfo/GTRSCE_files/pnsn20_GTSRCE.list'

In [3]:
path='/Users/banjo/Projects/nll_pnsn/bremerton'
ifile='/bremUpdated.hypo' 
# ifile='/or80.hypo'
# ofile = '/Users/banjo/Projects/nll_pnsn/util/stationInfo/GTRSCE_files/pnsn20_GTSRCE.list'
ofile = '/Users/banjo/Projects/nll_pnsn/util/stationInfo/GTRSCE_files/bremUp_GTSRCE.list'

In [4]:
datum = 0

In [5]:
# GOOD FOR ONE HYPO FILE
os.chdir(path)
ID=1
i=1
headers = []
PPicks = []
SPicks = []
MagPicks = []
evids = []
with open(path+ifile,'r') as fp:
    for line in fp.readlines():
        i=i+1
        if len(line) == 180:
            headers.append(line)
        elif len(line) == 121 or len(line)==120:            
            pha = line[14]
            if pha != ' ' and (line[11] == 'Z' or line[11] == '3'):
                PPicks.append([line])
            elif pha == 'P' and line[16] == '4':
                MagPicks.append(line)
            else:
                SPicks.append(line)
        else:
            evids.append(line)

In [6]:
# RUN THIS TO SORT A HYPOINVERSE FILE. 

# newHypo = np.concatenate([headers,PPicks,SPicks,MagPicks,evids])
# newHypo = np.concatenate([PPicks,SPicks])
# newHypo = str(newHypo)
# # bad_chars = ["\\n","'","[","]"]
# bad_chars = ["'[ ", "\\n", "']", "\\n","'","[","]"]


# for i in bad_chars:
#     newHypo = newHypo.replace(i,'')
    
# newHypo = newHypo.replace("\n ","\n")

# with open('/Users/banjo/Projects/nll_pnsn/PNSN_in_CVM_picks.out', 'w') as f:
#     f.write(newHypo.strip())
#     f.close()

## RUN EVERYTHING BELOW FOR A GTSRCE GENERATOR FOR THE FILE ABOVE. 

In [7]:
stns = []
for i in range(len(PPicks)):
    stn = PPicks[i][0][0:5]
    if stn[3] == " ":
        stn = PPicks[i][0][0:3]
        stns.append(stn)
    elif stn[4] ==" ":
        stn = PPicks[i][0][0:4]
        stns.append(stn)
    else:
        stn = PPicks[i][0][0:5]
        stns.append(stn)
#     stns.append(stn)

In [8]:
stas = np.unique(stns)

In [9]:
len(stas)

311

In [10]:
# len(stas)

In [11]:
path2 = '/Users/banjo/Projects/nll_pnsn/util/stationInfo'
file = '/all_stations.csv'
df1 = pd.read_csv(path2 + file)

In [12]:
astas = df1['sta'].values

In [13]:
# astas

In [14]:
# for i in range(len(astas)):
#     print(astas[i])

In [15]:
def Diff(li1, li2):
    return (list(list(set(li1)-set(li2)) + list(set(li2)-set(li1))))
# Driver Code
li1 = stas
li2 = astas
diff = Diff(li2, li1)
# print(diff)

In [16]:
# len(diff)

In [17]:
match_sta = []
match_lat =[]
match_lon=[]
match_ele=[]
# tmp = df2['sta'].values
tmp = stas

for i in range(len(tmp)):
    station = tmp[i]
    if station in diff:
        pass
    else:
#         print(station)
        match = df1.loc[df1['sta'] == station].values[0]
#         print(i,match)
        match_lat.append(match[2])
        match_lon.append(match[3])
        match_ele.append(match[4])
        match_sta.append(match[1])

In [18]:
# match_lat

In [19]:
df = pd.DataFrame(list(zip(match_sta,match_lat,match_lon,match_ele)),columns = ['Station','Lat','Lon','Elev'])

In [20]:
df.drop_duplicates()

,Station,Lat,Lon,Elev
0,7030,47.4509,-122.3022,0.122
1,7040,47.8559,-122.5833,0.008
2,ALCT,47.6469,-122.0377,0.055
3,ALKI,47.5751,-122.4176,0.001
4,ALLI,47.3201,-122.9157,0.077
...,...,...,...,...
305,WRW,47.8564,-120.8830,1.189
306,WYNO,47.4339,-123.6130,0.935
307,XTL,46.9298,-121.4945,1.665
308,YA2,46.5265,-120.5312,0.652


In [21]:
srcs = []
descriptor = []
zeros = []
lats = []
lons = []
elvs = []
names=[]
for i in range(len(match_lat)):
    lat = match_lat[i]
    lat = '%8.4f' %lat
    lon = match_lon[i]
    lon = '%8.4f' %lon
    ele = match_ele[i]
    ele = '%6.2f' %ele
    name = match_sta[i]
    name = '%5s' %name
    name = str(name)
    srcs.append('GTSRCE')
    descriptor.append('LATLON')
    zeros.append('0.0')
    lats.append(lat)
    names.append(name)
    lons.append(lon)
    elvs.append(ele)

In [22]:
# df = pd.DataFrame(list(zip(srcs,names,descriptor,lats,lons,zeros,elvs)),columns = ['Source','Station','Descriptor','Lat','Lon','Zeros','Elev'])

In [23]:
df = pd.DataFrame(list(zip(srcs,names,descriptor,lats,lons,zeros,zeros)),columns = ['Source','Station','Descriptor','Lat','Lon','Zeros','Elev'])

In [24]:
df = df.drop_duplicates()

In [25]:
df.to_csv(ofile,index=0,header = False,sep =' ')

In [25]:
# df

In [26]:
# diff

In [27]:
# len(diff)

In [28]:
# lat = df['Lat'].values

In [29]:
# df = df.sort_values('Lat')

In [30]:
# lat

In [31]:
# diff